In [1]:
!pip install pulp

You should consider upgrading via the '/opt/aapf/1.5.0/python_env/py3/bin/python -m pip install --upgrade pip' command.


In [2]:
from pulp import *
import pandas as pd

In [3]:
# ファイル読み込み (商品情報)
products = pd.read_csv("./products.csv")
display(products)

,商品名,価格(円),需要予測
0,スタンドカラーシャツ,1100,10
1,コットンオーバーシャツ,2200,10
2,オープンカラーシャツ,3080,20
3,ハイネックTシャツ,1100,10
4,リネンシャツ,2310,15
5,ドライTシャツ,1320,15
6,ボタンダウンシャツ,5060,30
7,トレイルシャツ,3410,20
8,ストライプシャツ,5060,10
9,アウトドアシャツ,1980,10


In [4]:
# ファイル読み込み (材料情報)
materials = pd.read_csv("./materials.csv")
display(materials)

,商品名,綿,麻,シルク,ウール,カシミヤ,アルパカ,ポリエステル,ポリウレタン,ナイロン,レーヨン,アクリル
0,スタンドカラーシャツ,0,0,0,0,0,46,10,0,0,0,0
1,コットンオーバーシャツ,40,10,0,0,0,0,0,0,0,0,0
2,オープンカラーシャツ,0,0,45,5,5,0,0,0,20,0,0
3,ハイネックTシャツ,0,0,0,0,20,0,0,10,0,30,30
4,リネンシャツ,0,50,0,0,0,0,0,0,0,0,0
5,ドライTシャツ,0,20,0,0,0,50,0,0,0,0,0
6,ボタンダウンシャツ,20,0,20,0,0,0,0,30,0,0,0
7,トレイルシャツ,20,0,0,0,0,0,0,10,0,0,0
8,ストライプシャツ,0,0,10,0,0,0,15,0,0,0,0
9,アウトドアシャツ,0,0,0,0,0,0,0,0,35,5,5


In [5]:
# ファイル読み込み (在庫情報)
stocks = pd.read_csv("./stocks.csv")
display(stocks)

,材料,在庫
0,綿,2000
1,麻,1000
2,シルク,500
3,ウール,2000
4,カシミヤ,100
5,ポリエステル,500
6,ポリウレタン,2000
7,ナイロン,2000
8,レーヨン,2200
9,アクリル,900


In [6]:
# 最適化オブジェクトの作成 (目的関数を最大化する)
prob = LpProblem(sense=LpMaximize)

In [7]:
# 変数定義(全て非負の整数)
# 変数は商品の数だけある
x = [LpVariable('x{}'.format(i), lowBound=0, upBound=products['需要予測'][i], cat=LpInteger) for i in range(products.shape[0])]
#x = [LpVariable('x{}'.format(i), lowBound=0, cat=LpInteger) for i in range(products.shape[0])]
print(x)

[x0, x1, x2, x3, x4, x5, x6, x7, x8, x9]


In [8]:
# 目的関数の設定
# 売上（商品価格×商品生産数の合計）
prob += pulp.lpDot(products['価格(円)'], x)

In [9]:
products['需要予測']

0    10
1    10
2    20
3    10
4    15
5    15
6    30
7    20
8    10
9    10
Name: 需要予測, dtype: int64

In [10]:
# 制約条件
# 生産で使用する原料　≦　原料の在庫
for material in materials.columns[1:]:
    prob += pulp.lpDot(materials.loc[:,material], x) <= stocks[stocks['材料'] == material]['在庫']

In [11]:
# 最適化問題を確認
print(prob)

NoName:
MAXIMIZE
1100*x0 + 2200*x1 + 3080*x2 + 1100*x3 + 2310*x4 + 1320*x5 + 5060*x6 + 3410*x7 + 5060*x8 + 1980*x9 + 0
SUBJECT TO
_C1: 40 x1 + 20 x6 + 20 x7 <= 2000

_C2: 10 x1 + 50 x4 + 20 x5 <= 1000

_C3: 45 x2 + 20 x6 + 10 x8 <= 500

_C4: 5 x2 <= 2000

_C5: 5 x2 + 20 x3 <= 100

_C6: 46 x0 + 50 x5 <= 0

_C7: 10 x0 + 15 x8 <= 500

_C8: 10 x3 + 30 x6 + 10 x7 <= 2000

_C9: 20 x2 + 35 x9 <= 2000

_C10: 30 x3 + 5 x9 <= 2200

_C11: 30 x3 + 5 x9 <= 900

VARIABLES
0 <= x0 <= 10 Integer
0 <= x1 <= 10 Integer
0 <= x2 <= 20 Integer
0 <= x3 <= 10 Integer
0 <= x4 <= 15 Integer
0 <= x5 <= 15 Integer
0 <= x6 <= 30 Integer
0 <= x7 <= 20 Integer
0 <= x8 <= 10 Integer
0 <= x9 <= 10 Integer



In [12]:
# 最適化実行
prob.solve()

# 最適値を確認
print('Optimal value = ', value(prob.objective))

# 目的関数の最適値を取る変数を確認
for v in prob.variables():
    print(v.name, '=', value(v))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/aapf/1.5.0/python_env/py3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/cebbedb02f6d41718b63a5da03b543f0-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/cebbedb02f6d41718b63a5da03b543f0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 16 COLUMNS
At line 72 RHS
At line 84 BOUNDS
At line 95 ENDATA
Problem MODEL has 11 rows, 10 columns and 25 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 301950 - 0.00 seconds
Cgl0004I processed model has 2 rows, 4 columns (4 integer (0 of which binary)) and 5 elements
Cutoff increment increased from 1e-05 to 220
Cbc0012I Integer solution of -301950 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective -301950, took 0 iterations and 0 nodes (0.00 

In [13]:
# 表に生産個数を追加して表示
tmp = {}
result = []
for v in prob.variables():
    v.name
    tmp[v.name] = value(v)
for i,product in enumerate(products['商品名']):
    result.append(tmp['x{}'.format(i)])
products['個数'] = result
display(products)

,商品名,価格(円),需要予測,個数
0,スタンドカラーシャツ,1100,10,0.0
1,コットンオーバーシャツ,2200,10,10.0
2,オープンカラーシャツ,3080,20,0.0
3,ハイネックTシャツ,1100,10,5.0
4,リネンシャツ,2310,15,15.0
5,ドライTシャツ,1320,15,0.0
6,ボタンダウンシャツ,5060,30,20.0
7,トレイルシャツ,3410,20,20.0
8,ストライプシャツ,5060,10,10.0
9,アウトドアシャツ,1980,10,10.0
